In [38]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

from winerror import XACT_E_TRANSACTIONCLOSED

In [4]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


## preprocessing

In [6]:
## Drop irrelevant columns
df = df.drop(['RowNumber','Surname','CustomerId'],axis=1)

In [7]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


In [8]:
## Encodecategorical variables
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,608,Spain,0,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,502,France,0,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,699,France,0,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,850,Spain,0,43.0,2,125510.82,1,NaN,1.0,79084.10,0


In [11]:
df['Geography'].value_counts()

Geography
France     5014
Germany    2510
Spain      2477
Name: count, dtype: int64

In [29]:
df.isnull().sum()
df.dropna(inplace=True)

In [30]:
##One Hot Encoding Geography
from sklearn.preprocessing import OneHotEncoder

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(df[['Geography']])

# Get feature names
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [33]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [34]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9993,1.0,0.0,0.0
9994,0.0,1.0,0.0
9995,0.0,1.0,0.0
9996,1.0,0.0,0.0


In [36]:
## Combine all the columns with original data
df = pd.concat([df.drop('Geography' ,axis=1),  geo_encoded_df], axis=1)

In [37]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,1.0,0.0,0.0
5,645.0,1.0,44.0,8.0,113755.78,2.0,1.0,0.0,149756.71,1.0,0.0,1.0,0.0


In [39]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl','wb')as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb')as file:
    pickle.dump(onehot_encoder_geo,file)

In [68]:
df.isnull().sum()
data = df.dropna()

In [69]:
data.isnull().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
dtype: int64

In [71]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0,1.0,0.0,0.0
1,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0,0.0,0.0,1.0
2,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0,1.0,0.0,0.0
3,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0,1.0,0.0,0.0
5,645.0,1.0,44.0,8.0,113755.78,2.0,1.0,0.0,149756.71,1.0,0.0,1.0,0.0


In [76]:
data.isnull().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
dtype: int64

In [72]:
## Dividing the data into Independent and Dependent
X = data.drop('Exited',axis=1)
y = data['Exited']

## Train test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##Scale these values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [73]:
X_train.shape

(7995, 12)

In [74]:
with open('scaler.pkl','wb')as file:
    pickle.dump(scaler,file)

# ANN Implementation

In [48]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [78]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [79]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [80]:
## compile the model
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [82]:
## Setup the tenserboard
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [83]:
## Set up the Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)

In [84]:
### Training the model
history = model.fit(X_train,
                    y_train,
                    validation_data=(X_test,y_test),
                    epochs=100,
                    batch_size=128,
                    callbacks=[tensorboard,early_stopping]
                    )

Epoch 1/100
63/63 [==============================] - 1s 4ms/step - loss: 0.5381 - accuracy: 0.7433 - val_loss: 0.4824 - val_accuracy: 0.7764
Epoch 2/100
63/63 [==============================] - 0s 2ms/step - loss: 0.4248 - accuracy: 0.8184 - val_loss: 0.4419 - val_accuracy: 0.7974
Epoch 3/100
63/63 [==============================] - 0s 3ms/step - loss: 0.3987 - accuracy: 0.8375 - val_loss: 0.4232 - val_accuracy: 0.8124
Epoch 4/100
63/63 [==============================] - 0s 2ms/step - loss: 0.3803 - accuracy: 0.8482 - val_loss: 0.4064 - val_accuracy: 0.8229
Epoch 5/100
63/63 [==============================] - 0s 2ms/step - loss: 0.3663 - accuracy: 0.8540 - val_loss: 0.3931 - val_accuracy: 0.8249
Epoch 6/100
63/63 [==============================] - 0s 2ms/step - loss: 0.3570 - accuracy: 0.8585 - val_loss: 0.3894 - val_accuracy: 0.8259
Epoch 7/100
63/63 [==============================] - 0s 2ms/step - loss: 0.3502 - accuracy: 0.8583 - val_loss: 0.3815 - val_accuracy: 0.8314
Epoch 8/100
6

In [77]:
print(y.unique())     # Should be [0, 1] for binary
print(y.isnull().sum())  # Should be 0

[1. 0.]
0


In [85]:
model.save('model.h5')

D:\ML\ann-project@dl\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [86]:
### LoadTenorboard extension
%load_ext tensorboard

In [90]:
# %tensorboard --logdir logs/fit/20250621-235325